<a href="https://colab.research.google.com/github/HE-Yangmei/Final_Prj/blob/master/cnn/transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04图像风格迁移

In [0]:
## !pip install scipy

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}



E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131323 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!ls

adc.json  drive  sample_data


In [0]:
import os
os.chdir("drive/Colab_Notebooks") 
os.chdir("final_prj/CNN")

In [0]:
## 修改环境变量
import os
os.environ['KERAS_BACKEND']='tensorflow'

from __future__ import print_function
from keras.preprocessing.image import load_img, img_to_array
from scipy.misc import imsave
import numpy as np
from scipy import optimize
from scipy.optimize import fmin_l_bfgs_b
## 拟牛顿最优化
import time
import argparse

from keras.applications import vgg19
from keras import backend as K

In [0]:
parser = argparse.ArgumentParser(description='Neural style transfer with Tensorflow.')
parser.add_argument('base_image_path', metavar='base', type=str,
                    help='Path to the image to transform.')
parser.add_argument('style_reference_image_path', metavar='ref', type=str,
                    help='Path to the style reference image.')
parser.add_argument('result_prefix', metavar='res_prefix', type=str,
                    help='Prefix for the saved results.')
parser.add_argument('--iter', type=int, default=10, required=False,
                    help='Number of iterations to run.')
parser.add_argument('--content_weight', type=float, default=0.025, required=False,
                    help='Content weight.')
parser.add_argument('--style_weight', type=float, default=1.0, required=False,
                    help='Style weight.')
parser.add_argument('--tv_weight', type=float, default=1.0, required=False,
                    help='Total Variation weight.')



_StoreAction(option_strings=['--tv_weight'], dest='tv_weight', nargs=None, const=None, default=1.0, type=<class 'float'>, choices=None, help='Total Variation weight.', metavar=None)

In [0]:
! ls

style  transfer.ipynb


In [0]:
args = parser.parse_args([r'content/y_rose.jpg', 
                         'style/rose.jpg', 
                         'rose_'])
base_image_path = args.base_image_path
style_reference_image_path = args.style_reference_image_path
result_prefix = args.result_prefix
iterations = args.iter

total_variation_weight = args.tv_weight
style_weight = args.style_weight
content_weight = args.content_weight

width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)


In [0]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size = (img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, img_nrows, img_cols))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((img_nrows, img_ncols, 3))
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image
                                  (style_reference_image_path))

if K.image_data_format() == 'channels_first':
    combination_image = K.placeholder((1, 3, img_nrows, img_ncols))
else:
    combination_image = K.placeholder((1, img_nrows, img_ncols, 3))
    
input_tensor = K.concatenate([base_image, 
                             style_reference_image, 
                             combination_image], axis = 0)

model = vgg19.VGG19(input_tensor = input_tensor, 
                   weights = "imagenet", include_top = False)
print('Model Loaded')

Model Loaded


In [0]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

## 效用函数
def gram_matrix(x):
    assert K.ndim(x) == 3
    if K.image_data_format() == 'channels_first':
        features = K.batch_flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

def content_loss(base, combination):
    return K.sum(K.square(combination - base))

def total_variation_loss(x):
    assert K.ndim(x) == 4
    if K.image_data_format() == 'channels_first':
        a = K.square(x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, 1:, :img_ncols - 1])
        b = K.square(x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, :img_nrows - 1, 1:])
    else:
        a = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
        b = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

## 标量
loss = K.variable(0.)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(base_image_features, 
                                     combination_features)
features_layers = ['block1_conv1', 'block2_conv1', 
                  'block3_conv1', 'block4_conv1', 
                  'block5_conv1']
for layer_name in features_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    s1 = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(features_layers)) * s1
loss += total_variation_weight * total_variation_loss(combination_image)

## 梯度
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

def eval_loss_and_grads(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((1, 3, img_nrows, img_ncols))
    else:
        x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

class Evaluator(object):
    
    def __init__(self):
        self.loss_value = None
        self.grads_values = None
        
    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value
    
    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values
    
evaluator = Evaluator()

x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(), 
                                    fprime = evaluator.grads, maxfun = 20)
    print('Current loss value:', min_val)
    
    img = deprocess_image(x.copy())
    fname = result_prefix + '_%d.png' % i
    imsave(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed  in %ds' % (i, end_time - start_time))
        


Start of iteration 0
Current loss value: 2277602800.0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:110: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Image saved as rose__at_iteration_0.png
Iteration 0 completed  in 24s
Start of iteration 1
Current loss value: 1584615200.0
Image saved as rose__at_iteration_1.png
Iteration 1 completed  in 19s
Start of iteration 2
Current loss value: 1391573400.0
Image saved as rose__at_iteration_2.png
Iteration 2 completed  in 20s
Start of iteration 3
Current loss value: 1294483200.0
Image saved as rose__at_iteration_3.png
Iteration 3 completed  in 19s
Start of iteration 4
Current loss value: 1222144300.0
Image saved as rose__at_iteration_4.png
Iteration 4 completed  in 20s
Start of iteration 5
Current loss value: 1178116600.0
Image saved as rose__at_iteration_5.png
Iteration 5 completed  in 20s
Start of iteration 6
Current loss value: 1145241300.0
Image saved as rose__at_iteration_6.png
Iteration 6 completed  in 19s
Start of iteration 7
Current loss value: 1119529600.0
Image saved as rose__at_iteration_7.png
Iteration 7 completed  in 19s
Start of iteration 8
Current loss value: 1096263700.0
Image sa

In [0]:
tensorboard --logdir=samples_starry

SyntaxError: ignored